# 把这文件放在和mlearn/同一层文件夹运行
## 不然import会报错

In [1]:
import mlearn
from mlearn import functional as F
from mlearn import layers
from mlearn.optimizers import SGD, RMSProp, Momentum
from torchvision import datasets,transforms
import torch
import numpy as np
import matplotlib.pyplot as plt
from mlearn import DataLoader
from mlearn import pre as P
import sys

# 加载数据集

In [2]:
train = datasets.MNIST("datasets", train=True, download=True,
                      transform = transforms.Compose([transforms.ToTensor()]))

test = datasets.MNIST("datasets", train=False, download=True,
                     transform = transforms.Compose([transforms.ToTensor()]))

pre = [P.normalize_MinMax]
trainset = DataLoader((train.data,train.targets),batch_size=32,shuffle=True,
                      preprocessing=pre)
testset = DataLoader((test.data, test.targets), batch_size=32, shuffle=True,
                    preprocessing=pre)

In [10]:
class Net(mlearn.Module):
    def __init__(self):
        self.dense1 = layers.Dense(784,10)
        self.dense2 = layers.Dense(300,150)
        self.dense3 = layers.Dense(150,10)
        
    def forward(self, inputs):
        o = self.dense1(inputs)
        o = F.relu(o)
        o = self.dense2(o)
        o = F.tanh(o)
        o = self.dense1(inputs)
        return ((o))
    



In [11]:
hist = []
net = Net()
optimizer = SGD(net,0.001)
EPOCHS = 1
net.zero_grad()
for epoch in range(EPOCHS):
    running_loss = 0.0
    bar = " "*20
    for i, batch in enumerate(trainset, 0):
        features, labels = batch
        net.zero_grad()
        predict = net(features.reshape(-1,784))
        loss = F.cross_entropy(predict, labels)
        loss.backward()
        optimizer.step()
        if i==3:
            raise RuntimeError("Stop")
        shapes = optimizer.step()
        running_loss += loss.data
        print(f"\r{epoch+1}/{EPOCHS} Batch %-4d/1874  [{bar}] -> Loss %.5f"%(i,loss.data), end="")
        sys.stdout.flush()
    print()
print('trainning completed!')


<function cross_entropy.<locals>.grad_fn at 0x7fc093bd3560>
<function _add.<locals>.grad_fn1 at 0x7fc093bd3050>
<function _matmul.<locals>.grad_fn2 at 0x7fc093bd30e0>
<function _add.<locals>.grad_fn2 at 0x7fc093bd3170>
(10,)
(784, 10)
<function cross_entropy.<locals>.grad_fn at 0x7fc093ca24d0>
<function _add.<locals>.grad_fn1 at 0x7fc093ca2680>
<function _matmul.<locals>.grad_fn2 at 0x7fc093ca2440>
<function _add.<locals>.grad_fn2 at 0x7fc093ca2320>
(10,)
(784, 10)
<function cross_entropy.<locals>.grad_fn at 0x7fc093ca2710>
<function _add.<locals>.grad_fn1 at 0x7fc093ca2830>
<function _matmul.<locals>.grad_fn2 at 0x7fc093ca2950>
<function _add.<locals>.grad_fn2 at 0x7fc093ca2560>
(10,)
(784, 10)
<function cross_entropy.<locals>.grad_fn at 0x7fc093ca2440>
<function _add.<locals>.grad_fn1 at 0x7fc093ca24d0>
<function _matmul.<locals>.grad_fn2 at 0x7fc093ca29e0>
<function _add.<locals>.grad_fn2 at 0x7fc093ca2680>
(10,)
(784, 10)


RuntimeError: Stop

In [5]:
correct = 0
total = 0

for batch in testset:
    features, labels = batch
    o = net(features.reshape(-1,784))
    predict = []
    for x in o.data:
        predict.append(np.argmax(x))
    for b in predict == labels.data:
        if b:
            correct += 1
    total += 32
print("Accuracy %.5f"%(correct / total))

Accuracy 0.13019


In [6]:
np.zeros_like((1,2))

array([0, 0])

In [7]:

num_episodes = 60000
for i_episode in range(1, num_episodes + 1):
    # Print out which episode we're on, useful for debugging.
    if i_episode % 1000 == 0:
        print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
        sys.stdout.flush()


Episode 60000/60000.

In [8]:
import time
import sys

toolbar_width = 40

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1)) # return to start of line, after '['

for i in range(toolbar_width):
    time.sleep(0.1) # do real work here
    # update the bar
    sys.stdout.write("-")
    sys.stdout.flush()

sys.stdout.write("]\n") # this ends the progress bar

[                                        ----------------------------------------]
